In [2]:
from pyscipopt import Model, quicksum

In [31]:
# create model - perfect matching
model = Model("Matching Problem")

# parameters for matching problem
workers = ['W1', 'W2', 'W3', 'W4']
jobs = ['J1', 'J2', 'J3', 'J4']

bipartite_graph = [['W1','J1'],
                   ['W2','J1'],
                   ['W2','J2'],
                   ['W3','J3'],
                   ['W3','J4'],
                   ['W4','J1'],
                   ['W4','J2'],
                   ['W4','J4']]

# create decision variables
x = {}
for wid, wvalue in enumerate(workers):
    for jid, jvalue in enumerate(jobs):
        if [wvalue, jvalue] in bipartite_graph:
            x[wid, jid] = model.addVar(lb=0, ub=1, vtype="B", name="x(%s,%s)"%(wid,jid))

In [32]:
# create objective function
model.setObjective(quicksum(x[i,j]for i, ivalue in enumerate(workers) for j, jvalue in enumerate(jobs) if [ivalue,jvalue] in bipartite_graph), sense='minimize')

# create assignment constraints1 (one worker assigned one job)
for wid, wvalue in enumerate(workers):
    model.addCons(quicksum(x[wid,jid] for jid, jvalue in enumerate(jobs) if [wvalue,jvalue] in bipartite_graph) == 1)

# create assignment constraints2 (each job at least assigned to one worker)
for jid, jvalue in enumerate(jobs):
    model.addCons(quicksum(x[wid,jid] for wid, wvalue in enumerate(workers) if [wvalue,jvalue] in bipartite_graph) >= 1)

In [33]:
model.optimize()
status = model.getStatus()
if status == "optimal":
    for wid, wvalue in enumerate(workers):
        for jid, jvalue in enumerate(jobs):
            if [wvalue,jvalue] in bipartite_graph and model.getVal(x[wid,jid]) > 0:
                print(f"{x[wid,jid]} = {model.getVal(x[wid,jid])}")

x(0,0) = 1.0
x(1,1) = 1.0
x(2,2) = 1.0
x(3,3) = 1.0
presolving:
(round 1, fast)       8 del vars, 7 del conss, 0 add conss, 3 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (2 rounds: 2 fast, 1 medium, 1 exhaustive):
 10 deleted vars, 8 deleted constraints, 0 added constraints, 3 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +4.00000000000000e+00 (1 solutions)
Dual Bound         : +4.00000000000000e+00
Gap                : 0.00 %
